In [1]:
! pip install chess

import chess
from random import randint,choice

from torch.utils.data import Dataset,DataLoader,SubsetRandomSampler
from sklearn.model_selection import train_test_split
import numpy as np 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 KB 3.3 MB/s eta 0:00:00


create multiple dataset classes, for different training tasks:

 -- learn the next word - autoregressive task (if the model is to be a decoder based)


 For each of these we may have to modify the last linear layer of the model (to match dimensionality of the output) So maybe it should not be part of ChessEncoder but outside of it. 

the model needs to be changed to be a decoder model only - https://datascience.stackexchange.com/questions/65241/why-is-the-decoder-not-a-part-of-bert-architecture

https://arxiv.org/pdf/2204.05832.pdf#:~:text=Decoder%2Donly%20models%20process%20a,decoder%20processes%20only%20the%20target.


google PaLM: https://arxiv.org/pdf/2204.02311.pdf


In [2]:

class LegalMovesBase(object):

  def __init__(self,min_moves:int=20,max_moves:int=100):
    self.min_moves = min_moves
    self.max_moves = max_moves

  def __iter__(self):
    return self

  def __next__(self):
    game_len = randint(self.min_moves,self.max_moves)
    board = chess.Board()
    tick = 0 
    game = []
    while tick<=game_len:
      move=board.lan(choice(list(board.legal_moves)))
      board.push_san(move)
      game.append(move)
      tick+=1
    game.extend(['#']*(self.max_moves-game_len-1))

    return game



In [ ]:
dataset = LegalMovesBase()

valid_test_split = 0.4
random_seed= 31
batch_size = 20
data_size = 20e6

# dividing training set to 0.6 of the total dataset
train_idx, valid_test_idx = train_test_split(np.arange(data_size),test_size=valid_test_split,shuffle=True)

# dividing validation and test set to sets of equal size, i.e. each 0.2 of the total dataset
valid_idx, test_idx = train_test_split(valid_test_idx,test_size=0.5,shuffle=True)


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,sampler=valid_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size,sampler=test_sampler)



In [3]:
dataset = LegalMovesBase()


In [6]:
next(iter(dataset))[:10]

['Nb1-a3',
 'f7-f5',
 'f2-f4',
 'Ng8-f6',
 'Ng1-h3',
 'd7-d5',
 'Na3-b5',
 'Bc8-d7',
 'Ke1-f2',
 'Nb8-c6']